# Functions for the ML
## Functions for setup


In [37]:
import requests 
from bs4 import BeautifulSoup


req = requests.get('https://www.worldometers.info/coronavirus/')
soup = BeautifulSoup(req.text)

## functions for the columns

In [21]:
## Dict Replace
ecology_dict = {
    "poor":1,
    "no data":0,
    "good":2,
    "excellent":4,
    "satisfactory":3
}
#data.ecology = data.ecology.replace(to_replace=ecology_dict)

In [28]:
### Finding OBJ columns

# obj = meth.loc[:,meth.dtypes == object]

In [22]:
def fix_user(user):
    if user == '#error':
        return ""
    else:
        return user[user.find('u'):]
#log.user_id = log.user_id.apply(fix_user)

In [29]:
## Gender replacement
# meth_quant['guardian'].replace({'mother':0, 'father':1},inplace=True)

In [31]:
### COrr Score

#meth_quant.corr().score

In [3]:
### a function for scores evaluation for get_mae function
'''candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
# Write loop to find the ideal tree size from candidate_max_leaf_nodes
scores = {leaf_size: get_mae_randf(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
print(scores)

# Store the best value of max_leaf_nodes (it will be either 5, 25, 50, 100, 250 or 500)
best_tree_size = min(scores, key=scores.get)'''

'candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]\n# Write loop to find the ideal tree size from candidate_max_leaf_nodes\nscores = {leaf_size: get_mae_randf(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}\nprint(scores)\n\n# Store the best value of max_leaf_nodes (it will be either 5, 25, 50, 100, 250 or 500)\nbest_tree_size = min(scores, key=scores.get)'

In [4]:
## column transformation
def dollar_to_number(x):
    if x == '$':
        return 1
    if x == '$$-$$$':
        return 2
    if x == '$$$$':
        return 3
    else:
        return 1
# data4['Price Range'] = data['Price Range'].apply(dollar_to_number)

In [18]:
############unification of samples
#df_train['sample'] = 1 #train
#df_test['sample'] = 0 #test
#df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

#data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

############separation afterwards

#train_data = data.query('sample == 1').drop(['sample'], axis=1)
#test_data = data.query('sample == 0').drop(['sample'], axis=1)

#y = train_data.Rating.values            # наш таргет
#X = train_data.drop(['Rating'], axis=1)

In [17]:
# выполним очистку от лишних символов
#data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: x.replace('[', ''))
#data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: x.replace(']', ''))

### Time columns

In [23]:
## RE SEARCH
import re

def sep_date(date):
    result = re.search(r'\ ', date)
    if result:
        return result.group(0)
    else:
        return ''
#log['date'] = log.time.apply(sep_date)

In [12]:
### РАзберемся со временем
# data['empty_Reviews'] = (data['Reviews']=='[[], []]').astype('float64')

# анализ тестовой базы выявил два пропуска, несмотря на то, что pandas.profiling на тренировочной базе пропусков не выявил, заполним их '[[], []]' и закинем в empty_Reviews
# data['Reviews'] = data['Reviews'].fillna('[[], []]')
# data['empty_Reviews'] = (data['Reviews']=='[[], []]').astype('float64')

In [13]:
#data['date_of_Review'] = data['Reviews'].str.findall('\d+/\d+/\d+')
## finding review dates

In [ ]:
# data['len_date'] = data['date_of_Review'].apply(lambda x: len(x)) ## length from date

In [ ]:
# data['len_date'].date_of_Review = data[data['len_date']==3].date_of_Review.apply(lambda x: x.pop(0))

In [ ]:
# data['one_Review'] = (data['len_date']==1).astype('float64') ## restaurants with only one review

In [14]:
def time_to_now(row):
    if row['date_of_Review'] == []:
        return None
    return datetime.datetime.now() - pd.to_datetime(row['date_of_Review']).max()

def time_between_Reviews(row):
    if row['date_of_Review'] == []:
        return None
    return pd.to_datetime(row['date_of_Review']).max() - pd.to_datetime(row['date_of_Review']).min()

#data['day_to_now'] = data.apply(time_to_now, axis = 1).dt.days
#data['day_between_Reviews'] = data[data['len_date']==2].apply(time_between_Reviews, axis = 1).dt.days

In [15]:
### Заполняем пропуски
#data['Number of Reviews'].fillna(0, inplace=True)

## Functions for Metrics

In [35]:
def get_stat_dif(column):
    cols = cacao.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(cacao.loc[cacao.loc[:, column] == comb[0], 'rating'], 
                        cacao.loc[cacao.loc[:, column] == comb[1], 'rating']).pvalue \
            <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break
            
'''for col in ['maker', 'origin', 'location', 'bean_type', 'bean_origin']:
    get_stat_dif(col)'''

"for col in ['maker', 'origin', 'location', 'bean_type', 'bean_origin']:\n    get_stat_dif(col)"

In [36]:
## get MAE for N amount of leaves
def get_mae_randf(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=42)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

## Functions for Pipelines

In [6]:
# We create the preprocessing pipelines for both numeric and categorical data.
'''numeric_features = ['Ranking','Price Range','Number of Reviews']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])''' ###--- NUMERIC

'''categorical_features = ['City']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])''' ###--- CATEGORICAL

"categorical_features = ['City']\ncategorical_transformer = Pipeline(steps=[\n    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),\n    ('onehot', OneHotEncoder(handle_unknown='ignore'))])"

In [7]:
# Preprocessor
'''preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])'''

"preprocessor = ColumnTransformer(\n    transformers=[\n        ('num', numeric_transformer, numeric_features),\n        ('cat', categorical_transformer, categorical_features)])"

In [9]:
## Full model (clf.fit)
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
'''clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs'))])'''

"clf = Pipeline(steps=[('preprocessor', preprocessor),\n                      ('classifier', LogisticRegression(solver='lbfgs'))])"

## EDA

### Graphs

In [19]:
# CORR
'''data_temp = data.loc[data['sample'] == 1, list(feat_importances.nlargest(15).index[0:15])]
plt.rcParams['figure.figsize'] = (12,6)
ax = sns.heatmap(data_temp.corr(), annot=True, fmt='.2g')
i, k = ax.get_ylim()
ax.set_ylim(i+0.5, k-0.5)'''

"data_temp = data.loc[data['sample'] == 1, list(feat_importances.nlargest(15).index[0:15])]\nplt.rcParams['figure.figsize'] = (12,6)\nax = sns.heatmap(data_temp.corr(), annot=True, fmt='.2g')\ni, k = ax.get_ylim()\nax.set_ylim(i+0.5, k-0.5)"

In [32]:
### BoxPlots
def get_boxplot(column):
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=column, y='score', 
                data=meth_quant,
               ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()
'''
for col in meth_quant.columns:
    get_boxplot(col)
'''

'\nfor col in meth_quant.columns:\n    get_boxplot(col)\n'

In [24]:
#plot
'''c = 0
for test in test_models:
    plt.plot(test, label=models_list[c])
    c += 1
    
plt.plot(y_test.iloc[0], label = "Real Data")
plt.legend()'''

'c = 0\nfor test in test_models:\n    plt.plot(test, label=models_list[c])\n    c += 1\n    \nplt.plot(y_test.iloc[0], label = "Real Data")\nplt.legend()'

In [25]:
###Measuring Error
##Metrics

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

'''metrics_list = ("mae", "mse", "msle")
c = 0
for test in test_models:
    
    mae = mean_absolute_error(test, y_test.iloc[0])
    mse = mean_squared_error(test, y_test.iloc[0])
    msle = mean_squared_log_error(test, y_test.iloc[0])
    print(f"{models_list[c]} errors:  MAE-{mae}, MSE-{mse}, msle-{msle}")
    c += 1
'''

'metrics_list = ("mae", "mse", "msle")\nc = 0\nfor test in test_models:\n    \n    mae = mean_absolute_error(test, y_test.iloc[0])\n    mse = mean_squared_error(test, y_test.iloc[0])\n    msle = mean_squared_log_error(test, y_test.iloc[0])\n    print(f"{models_list[c]} errors:  MAE-{mae}, MSE-{mse}, msle-{msle}")\n    c += 1\n'

In [20]:
###corr2
import seaborn as sns

'''corr = train.drop(['violation_zip_code', 'payment_amount', 
                      'clean_up_cost', 
                      'violation_street_number'], axis=1).\
                dropna(axis=1).\
                corr()
cmap = sns.diverging_palette(5, 250, as_cmap=True)'''

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
'''
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())'''

'\ncorr.style.background_gradient(cmap, axis=1)    .set_properties(**{\'max-width\': \'80px\', \'font-size\': \'10pt\'})    .set_caption("Hover to magify")    .set_precision(2)    .set_table_styles(magnify())'

### IQR

In [ ]:
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))[0]
# outliers_iqr(train.balance_due).max() - outliers_iqr(train.balance_due).min()

In [34]:
# IQR GRAPH
'''median = cacao.cocoa_percent.median()
IQR = cacao.cocoa_percent.quantile(0.75) - cacao.cocoa_percent.quantile(0.25)
print(median, IQR, median - 1.5*IQR, median + 1.5*IQR)
cacao.cocoa_percent.loc[cacao.cocoa_percent.between(median - 1.5*IQR, median + 1.5*IQR)].hist(bins = 16
                                                                                              , range = (40, 100), 
                                                                                             label = 'IQR')
cacao.cocoa_percent.loc[cacao.cocoa_percent <= 100].hist(alpha = 0.5, bins = 16, range = (40, 100),
                                                        label = 'Здравый смысл')
plt.legend();'''

"median = cacao.cocoa_percent.median()\nIQR = cacao.cocoa_percent.quantile(0.75) - cacao.cocoa_percent.quantile(0.25)\nprint(median, IQR, median - 1.5*IQR, median + 1.5*IQR)\ncacao.cocoa_percent.loc[cacao.cocoa_percent.between(median - 1.5*IQR, median + 1.5*IQR)].hist(bins = 16\n                                                                                              , range = (40, 100), \n                                                                                             label = 'IQR')\ncacao.cocoa_percent.loc[cacao.cocoa_percent <= 100].hist(alpha = 0.5, bins = 16, range = (40, 100),\n                                                        label = 'Здравый смысл')\nplt.legend();"

## Cross Validation

#### gridsearchCV

In [27]:
##CROSS VALIDATION
'''Gmodel = MLPRegressor(random_state = 42)
param_grid = {
    "max_iter" : [500,1000,2000],
    "activation" : ["logistic", "relu"]
}

GS = GridSearchCV(Gmodel, param_grid, scoring = "neg_mean_absolute_error", cv=3)'''
# best_model = GS.best_estimator_

'Gmodel = MLPRegressor(random_state = 42)\nparam_grid = {\n    "max_iter" : [500,1000,2000],\n    "activation" : ["logistic", "relu"]\n}\n\nGS = GridSearchCV(Gmodel, param_grid, scoring = "neg_mean_absolute_error", cv=3)'